# Hyperspectral CNN

This code is to train the Hyperspectral CNN. Warning: You need at least 18GB of RAM, to process the TfRecords.

In [1]:
cd ..

/home/sallinen/Programmation/predicting-poverty-through-time/src


In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

from lib.tfrecordhelper import TfrecordHelper
from sklearn.mixture import GaussianMixture as GMM
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
from torchvision import transforms
from tqdm.notebook import tqdm

import copy
import numpy as np
import os
import pandas as pd
import time
import torch
import torch.nn as nn
import torch.optim as optim

## Load data and preprocess

In [3]:
def load_dataset(path: str):
    """
    Helper to load dataset

    Args:
    - path (str): Path to dataset

    Returns:
    - dic which contains all data
    """
    tf_helper = TfrecordHelper(path, ls_bands="ms", nl_band="viirs")
    input_dic = {}
    tf_helper.keyword_lat = "lat"
    tf_helper.keyword_lon = "lon"
    tf_helper.process_dataset()
    for i, feature in enumerate(tf_helper.dataset):
        input_dic[i] = {
        "year": feature["years"].numpy(),
        "cluster_lat": feature["locs"].numpy()[0],
        "cluster_lon": feature["locs"].numpy()[1],
        "img": (feature["images"][:,:,:7].numpy()),
        "nightlight": np.mean(feature["images"][:,:,7].numpy()),
    }
    
    # Remove data where entry is broken (one channel contains only zeros)
    remove = []
    for feature in tqdm(input_dic):
        if input_dic[feature]["nightlight"] == 0:
            remove.append(feature)
            continue
        for dim in input_dic[feature]["img"]:
            if not np.any(dim):
                remove.append(feature)
                break
    
    for r in remove:
        input_dic.pop(r)
    return input_dic

In [4]:
path = "../data/tfrecords/raw/"
files = os.listdir(path) # path to the processed tfrecords from the previous step

In [5]:
input_dics = [] # will contain all information
for file in files:
    raw_path = path + file
    data = load_dataset(raw_path)
    input_dics.append(data)

2022-11-30 20:13:32.365911: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-30 20:13:32.366942: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


  0%|          | 0/781 [00:00<?, ?it/s]

  0%|          | 0/669 [00:00<?, ?it/s]

  0%|          | 0/419 [00:00<?, ?it/s]

  0%|          | 0/503 [00:00<?, ?it/s]

...

  0%|          | 0/645 [00:00<?, ?it/s]

  0%|          | 0/1611 [00:00<?, ?it/s]

  0%|          | 0/710 [00:00<?, ?it/s]

  0%|          | 0/516 [00:00<?, ?it/s]

  0%|          | 0/525 [00:00<?, ?it/s]

In [6]:
X = []
y = []
years = []
lat = []
lon = []
for country in tqdm(input_dics):
    data = country
    for feature in data:
        years.append(data[feature]["year"])
        lat.append(data[feature]["cluster_lat"])
        lon.append(data[feature]["cluster_lon"])
        data[feature]["img"][:3,:,:] *=3 # RGB images to dark, got better performance by using it
        X.append(data[feature]["img"])
        y.append(data[feature]["nightlight"])
X = np.array(X)
y = np.array(y)

  0%|          | 0/10 [00:00<?, ?it/s]

In [7]:
means = [np.mean(X[:,i,:,:]) for i in range(7)]
stds = [np.std(X[:,i,:,:]) for i in range(7)]

In [8]:
preprocess = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=means, std=stds)
])

## Bins for nighttime images

In [9]:
def nightlights_to_class(data):
    """
    Data are labels. Perform GMM based on the input and creates 5 classes out of it.

    Args:
    - data: radiance (nighttime images)

    Return:
    - list of labels
    """
    x = data.reshape(-1,1)
    gmm = GMM(n_components=5).fit(x)
    labels = gmm.predict(x)
    cut_label1 = data[labels==0].max()
    cut_label2 = data[labels==1].max()
    cut_label3 = data[labels==2].max()
    cut_label4 = data[labels==3].max()
    cut_label5 = data[labels==4].max()
    cutoffs = [cut_label1, cut_label2, cut_label3,  cut_label4, cut_label5]
    cutoffs = sorted(cutoffs)
    
    y_labels = []
    for d in data:
        if d <= cutoffs[0]:
            y_labels.append(0)
        elif d <= cutoffs[1]:
            y_labels.append(1)
        elif d <= cutoffs[2]:
            y_labels.append(2)
        elif d <= cutoffs[3]:
            y_labels.append(3)
        else:
            y_labels.append(4)
    return np.array(y_labels)

In [10]:
y_labels = nightlights_to_class(y)

## Pytorch Dataset

In [32]:
class MyDataset(Dataset):
    def __init__(self, data, target, transform=None):
        self.data = data
        self.target = torch.from_numpy(target).long()
        self.transform = transform
        
    def __getitem__(self, index):
        x = self.data[index]
        y = self.target[index]
        if self.transform:
            x = self.transform(x) # transpose is required by PyTorch

        return x, y
    
    def __len__(self):
        return len(self.data)

In [33]:
dataset = MyDataset(X, y_labels, preprocess)

In [34]:
loader = DataLoader(
    dataset,
    batch_size=128,
    shuffle=True,
    num_workers=2,
    pin_memory=torch.cuda.is_available()
)

In [35]:
indices = list(range(len(dataset)))
split = int(np.floor(.4 * len(dataset)))
train_indices, val_indices = indices[split:], indices[:split]
train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)
train_loader = torch.utils.data.DataLoader(dataset, batch_size=128, 
                                           sampler=train_sampler)
validation_loader = torch.utils.data.DataLoader(dataset, batch_size=128,
                                                sampler=valid_sampler)
dataloaders = {
    "train": train_loader,
    "val": validation_loader
}

dataset_sizes = {
    "train": len(train_sampler),
    "val": len(valid_sampler)
}

## CNN

In [36]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [37]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Best val Acc: {best_acc:4f}')

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [38]:
model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', pretrained=True) # load resnet

Using cache found in /home/sallinen/.var/app/com.jetbrains.PyCharm-Professional/cache/torch/hub/pytorch_vision_v0.10.0
/home/sallinen/miniconda3/envs/predicting-poverty-through-time/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/sallinen/miniconda3/envs/predicting-poverty-through-time/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Hyperspectral Setting

In [39]:
new_input = nn.Conv2d(7, 64, kernel_size=(7,7), stride=(2,2), padding=(3,3), dilation=1, bias=False)
model.conv1 = new_input

Modify outputs

In [40]:
model_ft = model
num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_ftrs, 5)

In [41]:
model_ft = model_ft.to(device)

In [42]:
criterion = nn.CrossEntropyLoss()
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,num_epochs=19)

Epoch 0/18
----------
train Loss: 1.1592 Acc: 0.5138
val Loss: 1.0501 Acc: 0.5261

Epoch 1/18
----------
train Loss: 0.9543 Acc: 0.5873
val Loss: 1.1457 Acc: 0.4486

Epoch 2/18
----------
train Loss: 0.8960 Acc: 0.6116
val Loss: 1.0682 Acc: 0.5375

Epoch 3/18
----------
train Loss: 0.8472 Acc: 0.6314
val Loss: 1.1126 Acc: 0.4442

Epoch 4/18
----------
train Loss: 0.7948 Acc: 0.6556
val Loss: 1.1716 Acc: 0.4710

Epoch 5/18
----------
train Loss: 0.7653 Acc: 0.6574
val Loss: 1.5955 Acc: 0.4008

Epoch 6/18
----------
train Loss: 0.7178 Acc: 0.6931
val Loss: 1.1291 Acc: 0.5158

Epoch 7/18
----------
train Loss: 0.6569 Acc: 0.7325
val Loss: 1.2063 Acc: 0.5048

Epoch 8/18
----------
train Loss: 0.6365 Acc: 0.7404
val Loss: 1.2779 Acc: 0.5165

Epoch 9/18
----------
train Loss: 0.6287 Acc: 0.7431
val Loss: 1.4686 Acc: 0.5327

Epoch 10/18
----------
train Loss: 0.6196 Acc: 0.7455
val Loss: 3.1947 Acc: 0.3997

Epoch 11/18
----------
train Loss: 0.6116 Acc: 0.7509
val Loss: 1.4220 Acc: 0.5386

Ep

In [43]:
torch.save(model.state_dict(), f'model_weights_all_countries_multichannel_{time.time()}.pth')

## Extract Weights

In [46]:
nmodel = torch.nn.Sequential(*list(model_ft.children())[:-1])

In [47]:
if torch.cuda.is_available():
    nmodel.to('cuda')

Forward pass

In [49]:
for data in input_dics:
    for feature in tqdm(data, total=len(data)):
        input_batch = preprocess(data[feature]['img']).unsqueeze(0)
        
        with torch.no_grad():
            output = nmodel(input_batch.to('cuda'))
        data[feature]["feature"] = np.squeeze(output.cpu())

  0%|          | 0/781 [00:00<?, ?it/s]

  0%|          | 0/669 [00:00<?, ?it/s]

  0%|          | 0/419 [00:00<?, ?it/s]

  0%|          | 0/479 [00:00<?, ?it/s]

  0%|          | 0/475 [00:00<?, ?it/s]

  0%|          | 0/645 [00:00<?, ?it/s]

  0%|          | 0/1588 [00:00<?, ?it/s]

  0%|          | 0/708 [00:00<?, ?it/s]

  0%|          | 0/516 [00:00<?, ?it/s]

  0%|          | 0/525 [00:00<?, ?it/s]

Merge of weights and dataframe

In [50]:
df = pd.DataFrame()
for data in input_dics:
    years = []
    lat = []
    lon = []
    features = []
    nightlights = []
    for feature in tqdm(data, total=len(data)):
        years.append(data[feature]["year"])
        lat.append(data[feature]["cluster_lat"])
        lon.append(data[feature]["cluster_lon"])
        features.append(data[feature]["feature"].numpy().tolist())
        nightlights.append(data[feature]["nightlight"])
    tmp = pd.DataFrame.from_dict({"year": years, "lat": lat, 'lon': lon, "features": features, "nightlight": nightlights})
    df = df.append(tmp)

  0%|          | 0/781 [00:00<?, ?it/s]

/tmp/ipykernel_373/3038713665.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(tmp)


  0%|          | 0/669 [00:00<?, ?it/s]

  0%|          | 0/419 [00:00<?, ?it/s]

  0%|          | 0/479 [00:00<?, ?it/s]

...

/tmp/ipykernel_373/3038713665.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(tmp)


  0%|          | 0/516 [00:00<?, ?it/s]

/tmp/ipykernel_373/3038713665.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(tmp)


  0%|          | 0/525 [00:00<?, ?it/s]

/tmp/ipykernel_373/3038713665.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(tmp)


In [51]:
df.to_csv("../data/cnn_features/resnet_trans_all_countries_hyper.csv", index=False)